# **Análise Exploratória de Dados do COVID-19 no Brasil**

### *Uma análise crítica ao tratamento de dados feito pelo Ministério da Saúde da República Federativa do Brasil*


### Índice:
* [Justificativa](#justificativa)
* [Fontes](#fonte)
* [Análise](#analise)
* [Conclusões](#conclusoes)
* [Referências](#referencias)

---

# Justificativa <a id='justificativa'></a>
Existe uma narrativa, de alguns governos estaduais de que o Brasil segue em ascensão exponencial em número de casos/óbitos.
E, argumentam, os gestores destes estados, que por este motivo é necessário a implementação de uma extensão das quarentenas.
A ALERJ, por exemplo, votou nesta quarta-feira (dia 20) um projeto de lei que impunha medidas restritivas de locomoção ainda maiores e prolongadas, na qual requeria-se uma extensão por mais 15 dias. O projeto de lei, entretanto fora rejeitado pela maioria dos Deputados Estaduais.

Todavia um estudo pela Singapore University of Technology, em meados de Abril demonstrou, mediante uma análise preditiva, de que o ciclo do Covid-19 atinge aproximadamente 80% do contágio se dá nos 2 primeiros meses (https://www.flasog.org/static/COVID-19/COVID19PredictionPaper20200426.pdf)

De acordo com o estudo, o ciclo do contágio forma um modelo gaussiano (Distribuição Normal Padrão), atingindo seu percentil 50 em aproximadamente 40 dias.

O Brasil, por sua vez, desde a constatação do primeiro caso em 16 de março até hoje, perfaz 2 meses de coleta de dados. E para nossa surpresa, percebemos um padrão dissonante com aquele apresentado pela Singapore University of Technology.

No site criado pelo governo para publicação dos dados relacionados à pandemia (https://covid.saude.gov.br/) vemos que o gráfico de número de óbitos por data de notificação segue uma forma de Distribuição assimétrica à esquerda ou assimétrica negativa (left skewed ou negative skewed), conforme modelo abaixo:

In [ ]:
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

x = np.linspace(0, 3, 60)
y = np.exp(x)
plt.style.use('ggplot')
f, ax = plt.subplots(figsize=(14,6))

plt.bar(x, y, align='center', width=0.03)




## Metodologia
A metodologia aplicada irá verificar se há diferença entre as datas da notificação dos óbitos com a efetiva data do seu acontecimento. Pois os dados do site de campanha (brazil_covid19.csv), na sua coluna "new_deaths", apresenta os dados de óbitos por data de notificação, o que pode vir a ser diferente dos números de mortes de acordo com a data do óbito. 

Para isto, faremos uso de um outro dataset, este disponibilizado pelos cartórios espalhados pelo Brasil, (https://data.brasil.io/dataset/covid19/_meta/list.html), o qual trás em sua coluna "new_deaths_covid19" a quantidade de óbitos ocorrida na respectiva data, conforme documentação:

    new_deaths_covid19: Quantidade de óbitos em decorrência de suspeita ou confirmação de covid19 para o estado state ocorridos na data date (em 2020).


# Fontes <a id='fonte'></a>
* Raphael Fontes (https://www.kaggle.com/unanimad/corona-virus-brazil)

* Ministério da Saúde (https://covid.saude.gov.br/)

* Secretarias Estaduais de Saúde (https://brasil.io/dataset/covid19/caso_full/?page=2)

# Análise <a id='analise'></a>
Primeiro iremos validar os dados do síte do Ministério da Saúde, os quais apresentam óbitos por data de notificação.

Posteriormente, iremos conferir se há diferença entre a data de notificação e a data efetiva do óbito.
Caso haja, iremos alocar os óbitos das datas de notificação, para a data efetiva do óbito, fazendo assim com que os dados apresentem uma realidade mais aproximada do cenário epidemiológico 


## 1) Análise das mortes pela data da notificação 

In [ ]:
# Imports
import pandas as pd
import matplotlib.dates as mdates 
import matplotlib.ticker as tkr
from datetime import datetime
import datetime as dt

# Dados de Óbitos das Secretarias Estaduais de Saúde do dia 12 de Junho
covid2 = pd.read_csv('../input/corona-virus-brazil/brazil_covid19.csv')
covid2 = covid2.sort_values(['state', 'region', 'date'])

# Create a new column with new deaths/day.
covid2['new_deaths'] = np.where(covid2.state.eq(covid2.state.shift()), covid2.deaths.diff(), 1)


In [ ]:
# Invert date format
covid2["date"] = pd.to_datetime(covid2["date"]).dt.strftime('%d/%m/%Y')


#### Temos aproximadamente 73 dias de dados inseridos, desde as primeiros óbitos confirmados Covid-19 no Brasil. 
#### Todavia, temos 1896 inputs.
#### Isto ocorre porque os dados são atualizados diariamente de forma repetida.

Para tanto, precisaremos realizar o agrupamento pela data ('date'), bem como agregar os números de mortes' (new_deaths') aplicando a soma de eventos repetidos no mesmo dia.

In [ ]:
# Group by day and Aggregate by adding each days` deaths 
covid2 = covid2.groupby('date').agg({'new_deaths': 'sum'})
covid2 = covid2.groupby('date').sum().reset_index()
covid2.head()


In [ ]:
# Transform dates into an array of tuples to include on y-axis
x = covid2['date']
xdates = [dt.datetime.strptime(i,'%d/%m/%Y') for i in x]
y = covid2['new_deaths'].values

# Grab last day and last 24hs deaths
now = datetime.now()
last_day = now.strftime('%d/%m')




In [ ]:

# Plot
plt.style.use('ggplot')
f, ax = plt.subplots(figsize=(16,8), dpi=300)

# Major ticks every 50, minor ticks every 10
major_ticks = np.arange(0, 1500, 200)
minor_ticks = np.arange(0, 1500, 50)

ax.set_yticks(major_ticks)
ax.set_yticks(minor_ticks, minor=True)

# And a corresponding grid
ax.grid(which='minor', alpha=0.5)
ax.grid(which='major', alpha=0.8)

plt.bar(xdates,y, width=0.7, label="Óbitos por data da Notificação")
plt.legend(loc="upper left")
plt.title("Mortos por COVID19 de acordo com a data da Notificação", fontweight='bold')
plt.ylabel('Óbitos', fontsize=10)
plt.setp(plt.gca().xaxis.get_majorticklabels(),rotation=90)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%d/%m'))
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=2))

f.text(0.5, 0.8, f'TOTAL: {int(covid2.new_deaths.sum())}', horizontalalignment='center', bbox={'alpha': 0.1, 'pad': 6})
f.text(0.12, 0.01, "Fonte: Secretarias Estaduais de Saúde {last_day}", fontsize=10)
f.text(0.9, 0.01, "Código: www.kaggle.com/fabianofpena/covid-19-no-brasil/", ha="right", fontsize=10)

plt.savefig('filename.png')
plt.show()


**Conforme apresentado pelo site do Ministério da Saúde, nesta primeira análise confirmamos que o gráfico de número de óbitos por data de notificação, se assemelha a um modelo de Distribuição assimétrica à esquerda ou assimétrica negativa (left skewed ou negative skewed).**

**Passemos, então, à segunda análise para verificar se há incompatibilidade entre as datas de notificação e a data efetiva do óbito.**

**Lembrando que:**
1. #### ***Se não houver diferença entre as datas de notificação e as datas efetivas dos óbitos***, podemos concluir que o ***Brasil segue um padrão atípico*** em relação aos demais países que tiveram seu período de expensão da epidemia por um período médio de 40 dias.

1. #### ***Caso haja diferença entre as datas de notificação e as datas efetivas dos óbitos***, e esta diferença seja suficiente para demonstrar que há a formação de um "bell-shaped curve", podemos concluir que se torna ***injustificável todas as medidas de prorrogação das medidas restitrivas impostas pelos governos ("lockdown").***


## 2) Análise das mortes pela data do Óbito 

Neste dataset, iremos carregar os ***dados do dia atual*** bem como do ***dia anteior*** a fim de constatar o percentual de casos óbitos atualizados nas últimas 24 horas.
Importante frisar que caso o dia 14 de maio (pico atual) venha permanecer como o ponto mais alto de óbitos acumulados em 24 horas, poderemos confirmar que a pandemia no Brasil já atingiu o percentil 50. O que significa dizer que estamos em declínio. 

In [ ]:
""" Dia 27 de Junho """
# Fonte: https://data.brasil.io/dataset/covid19/_meta/list.html
# Dados de Óbitos dos Cartórios
obitos2 = pd.read_csv("../input/cartoriosjun/obito_cartorio27jun.csv", sep=",", error_bad_lines=False)

# Check dtypes
obitos2.dtypes




**Verificando os dados, percebemos que a Series 'date' é um objeto. Portanto, devemos transforma-la em Datetime:**

In [ ]:
# Convert to Datetime
obitos2["date"] = pd.to_datetime(obitos2["date"]).dt.strftime('%d/%m/%Y')
obitos2['date'] = pd.to_datetime(obitos2.date, format='%d/%m/%Y')
obitos2.describe()

#### Temos aproximadamente 73 dias de dados inseridos, desde as primeiros óbitos confirmados Covid-19 no Brasil. 
 **Todavia, temos 9881 inputs.**
 **Isto ocorre porque os dados são atualizados diariamente e de forma cumulativa. **
 
**Para tanto, precisaremos realizar o agrupamento pela data ('date'), bem como agregar os números de mortes (new_deaths_covid19') aplicando a soma de eventos (óbitos) repetidos no mesmo dia.**

In [ ]:
# Group by day and Aggregate by adding each days` deaths 
agg2 = obitos2.groupby('date').agg({'new_deaths_covid19': 'sum'})
agg2 = agg2.groupby('date').sum().reset_index()


In [ ]:
# Drop days without death
agg2 = agg2[agg2.new_deaths_covid19 != 0.0]

# Filter by dates before 8th Mar
agg2 = agg2[agg2['date'] >= '2020-03-08']

print(agg2.isna().sum())
agg2.head()

In [ ]:
# Total accumulated deaths
agg2['new_deaths_covid19'].sum()

In [ ]:

""" Dia 26 de Junho """
# Dados de Óbitos dos Cartórios dia 26 de Junho
obitos = pd.read_csv("../input/cartoriosjun/obito_cartorio26jun.csv",
                   sep=",", error_bad_lines=False)


# Check dtypes
obitos.dtypes

# Convert to Datetime
obitos["date"] = pd.to_datetime(obitos["date"]).dt.strftime('%d/%m/%Y')
obitos['date'] = pd.to_datetime(obitos.date, format='%d/%m/%Y')


# Group by day and Aggregate by adding each days` deaths 
agg = obitos.groupby('date').agg({'new_deaths_covid19': 'sum'})
agg = agg.groupby('date').sum().reset_index()


#Drop days without death
agg = agg[agg.new_deaths_covid19 != 0.0]

# Filter by dates before 8th Mar
agg = agg[agg['date'] >= '2020-03-08']



Aqui vamos criar uma nova série ("coluna") com a diferença de óbitos atualizados nas últimas 24 horas, a fim de plotar em conjunto a ***distribuição dos óbitos totais mais recente*** com a quantidade de óbitos apuradas nas ***últimas 24 horas***. Com isto poderemos mensurar se ainda há muitos laudos em atraso após o dia 14 de maio. 
Desta feita, não havendo, e os dados das atualizados nas últimas 24 horas se concentrarem mais nos dias recentes, confirmaremos o pico no dia 14 de maio.

In [ ]:
# Extract last 24hs data
agg2['diff'] = agg2['new_deaths_covid19'].subtract(agg['new_deaths_covid19'], fill_value=0)


In [ ]:
# Check deaths confirmed in the last 24hs
agg2['diff'].sum()

In [ ]:
# Transform datetime Series into an object
agg2["date"] = pd.to_datetime(agg2["date"]).dt.strftime('%d/%m/%Y')


In [ ]:
# Grab axis and transform dates into an array of tuples
x = agg2['date']
xdates = [dt.datetime.strptime(i,'%d/%m/%Y') for i in x]
y = agg2['new_deaths_covid19'].values
y = y.astype(int)
y2 = agg2['diff'].values
y2 = y2.astype(int)


In [ ]:
# Grab last day and last 24hs deaths
now = datetime.now()
last_day = now.strftime('%d/%m')
last_deaths = int(agg2['diff'].sum())
prev_death = int(agg2['new_deaths_covid19'].sum()-agg2['diff'].sum())


In [ ]:
# Add Labels on the top of the bars
def bar_label(bars):
    for bar in bars:
        height = bar.get_height()
        if height == 0:
            continue
        ax.annotate('{}'.format(height),
            xy=(bar.get_x() + bar.get_width() / 2, bar.get_y() + height),
            xytext=(0, 3),  # 3 points vertical offset
            textcoords="offset points",
            ha='center', va='bottom', fontsize=8, rotation=90)

In [ ]:
# Plot
plt.style.use('ggplot')
f, ax = plt.subplots(figsize=(16,8), dpi=300)

# Texts
f.text(0.5, 0.8, f'TOTAL: {int(agg2.new_deaths_covid19.sum())}', horizontalalignment='center', bbox={'alpha': 0.1, 'pad': 6})
f.text(0.125, 0.01, f"Fonte: Portal Transparência Cartórios {last_day}", fontsize=10)
f.text(0.9, 0.01, "Código: www.kaggle.com/fabianofpena/covid-19-no-brasil/", ha="right", fontsize=10)

# Major ticks every 200, minor ticks every 100
major_ticks = np.arange(0, 1001, 200)
minor_ticks = np.arange(0, 1001, 50)

ax.set_yticks(major_ticks)
ax.set_yticks(minor_ticks, minor=True)

# And a corresponding grid
ax.grid(which='minor', alpha=0.4)
ax.grid(which='major', alpha=0.9)

plt.title("Óbitos por COVID-19")

# ax.plot(y[:len(ma)], ma, color="#dd0000")
ax.bar(xdates,y, width=0.7, color='darkorange', alpha=0.9, label="Óbitos Anteriores")
#ax.bar(xdates,list(y2.astype(int)), width=0.7, color='steelblue', bottom=y, label="Óbitos confirmados nas últimas 24 horas")
top_bar = ax.bar(xdates,list(y2), width=0.7, color='steelblue', bottom=y, label="Óbitos confirmados nas últimas 24 horas")
bar_label(top_bar)

ax.legend((f'Óbitos anteriores: {prev_death}',
           f"Óbitos Reportados no dia {last_day}: {last_deaths}"), loc='upper left')

plt.ylim(0,agg2.new_deaths_covid19.max()*1.2)
plt.ylabel('Óbitos', fontsize=12)
plt.xlabel('Data do Óbito', fontsize=12, labelpad=10)
plt.setp(plt.gca().xaxis.get_majorticklabels(),rotation=90)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%d/%m'))
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=2))

plt.show()

# Conclusões <a id='conclusoes'></a>


Conforme era de se esperar, a diferença entre os dados referentes à data da notificação do óbito e seu efetivo acontecimento, nos revela um novo modelo de distribuição semelhante àquele obtido no estudo realizado pela Singapore University of Technology em meados de abril, a saber, Distribuição Normal Padrão.

Por fim, deixo uma nota crítica aos trabalhos realizados por estatísticos da União e das diversas unidades federativas do Brasil, que por imprecisão e imperícia vêem legitimando, mediante suas análises, a tomada medidas bastante sensíveis pelos gestores estaduais, a ponto de afetar não apenas a economia do país mas diversos setores da sociedade.

Em vídeo recente (https://www.youtube.com/watch?v=OaCbcqLwg0c), Dra. Margaret Harris, Porta-Voz da OMS, disse:

    "Nunca dissemos façam lockdown — dissemos identifiquem, façam o rastreamento 
    e acompanhem o desenvolvimento dos casos, isolem e tratem os doentes"
(https://www.smh.com.au/politics/federal/who-backs-australian-move-to-ease-restrictions-20200426-p54na9.html)

Fica mais que evidente que a mudança de narrativa, vez por outra, por partes das autoridades internacionais e nacionais, demonstram um total desconhecimento (para não dizer que há uma conspiração coordenada) dos padrões analisados por diversos estudos que comprovam que a quarentena não produz efeitos eficazes a fim de "achatar a curva" de gauss.

Lado outro, já sabíamos, bem como estamos presenciando os efeitos nefastos de uma quarentena prolongada em todo o mundo.

Em um próximo estudo gostaria de analisar os estudos recentes da Universidade de Stanford que demonstra um índice de letalidade demasiadamente menor do que aquele que vem sendo propagado, o qual concluiu que a letalidade do Covid-19 chega a ser de aproximadamente 4 vezes menor do que a da influenza.

Mas por ora é isto.

# Referências <a id='referencias'></a>

[Singapore University of Technology] (https://www.flasog.org/static/COVID-19/COVID19PredictionPaper20200426.pdf).

[Ministério da Saúde] (https://data.brasil.io/dataset/covid19/_meta/list.html), (https://covid.saude.gov.br/)
